In [1]:
import pandas as pd
from tqdm import tqdm                               #progress bar
from binance.client import Client

In [2]:
client = Client()

In [3]:
info = client.get_exchange_info()

In [4]:
symbols = [x['symbol'] for x in info['symbols']]

In [5]:
#exclude leverage tokens
exclude = ['UP', 'DOWN', 'BEAR', 'BULL']

In [6]:
non_lev = [x for x in symbols if all(excludes not in x for excludes in exclude)]

In [7]:
#relevant symbols with USDT
relevant = [x for x in non_lev if x.endswith('USDT')]

In [8]:
#price of each "relevant" crypto
klines = {}

for symbol in tqdm(relevant):
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC')

100%|████████████████████████████████████████████████████████████████████████████████| 329/329 [01:33<00:00,  3.51it/s]


In [20]:
(pd.DataFrame(klines['BTCUSDT'])[4].astype(float).pct_change() + 1).prod() - 1

-0.004570741979002313

In [23]:
returns, symbols = [], []

for symbol in relevant:
    if len(klines[symbol]) > 0:
        cumret = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() + 1).prod() - 1
        returns.append(cumret)
        symbols.append(symbol)

In [26]:
retdf = pd.DataFrame(returns, index = symbols, columns = ['Return']) 
retdf

,Return
BTCUSDT,-0.004571
ETHUSDT,-0.004023
BNBUSDT,-0.001821
NEOUSDT,-0.001961
LTCUSDT,-0.000672
...,...
USTUSDT,0.000000
JOEUSDT,0.005468
ACHUSDT,0.005130
IMXUSDT,-0.003332


In [29]:
#Best performing cryptos
retdf.Return.nlargest(10)

CVXUSDT      0.053378
QUICKUSDT    0.040609
RADUSDT      0.036398
SYSUSDT      0.022918
OMGUSDT      0.022640
CHRUSDT      0.022596
DUSKUSDT     0.022532
DFUSDT       0.021382
RAMPUSDT     0.018963
AVAUSDT      0.015085
Name: Return, dtype: float64